In [68]:
import obspy, os
import pandas as pd
import numpy as np
from obspy.geodetics.base import gps2dist_azimuth

In [69]:
WD_new_lat_lon = pd.read_csv('WD_new.csv',names=['lon','lat'])
WD_bty = pd.read_csv('wd_ChaoJing_new.csv',header=None)
GPStime = pd.read_csv('gpstime.csv',names=['time'])
GPSlat = pd.read_csv('gps_lat.csv',names=['AUV','Buoy','Ship'])
GPSlon = pd.read_csv('gps_lon.csv',names=['AUV','Buoy','Ship'])

In [70]:
envfile = 'Buoy-Ship'

freq = 18500.000
Nmedia = 1 
topopt = 'CVW'
option2='A*'
cpbottom = 2300
csbottom = 0.00
den_bottom = 2.300
alpha_bottom = 1.200
Nbeams = 1001
alpha = 20.0
step = 1.000
zbox = 50.000

In [71]:
def createBTY(lentobty, Shipbty, sorted_Dis,sorted_Bty,qqq,runtype,rbox, savepath, envfile,texttime):
    if runtype == 'E':
        end = 'eigen'
    elif runtype =='A':
        end = 'amp'
    ##Create bty file 
    cmd = ''' echo $'\"C\"' > %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd) 
    cmd = '''echo %(lentobty)s  >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd) 
    Shipbty = abs(Shipbty)
    if Shipbty> max(abs(sorted_Bty)):
        Shipbty = max(abs(sorted_Bty))
    cmd = ''' echo 0.000000 %(Shipbty)f >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd)
    for i in range(lentobty-2):
        j = i*qqq
        dist = sorted_Dis[j]
        bty  = -1*sorted_Bty[j]
        cmd  = ''' echo %(dist)f %(bty)f >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty ''' %locals()
        os.system(cmd)
    cmd = ''' echo %(rbox)f %(bty)f >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd)
    cmd = ''' echo  >> %(savepath)s/%(envfile)s_time%(texttime)s_%(end)s.bty''' % locals()
    os.system(cmd)

In [72]:
def createENV(savepath, envfile,texttime, titleenv,freq,Nmedia, topopt, BtyLine,option2,cpbottom, csbottom, den_bottom,alpha_bottom, \
        NSD,SD,NRD,RD,NRR,RR,runtype,Nbeams,alpha,step,zbox,rbox):
    if runtype == 'E':
        end = 'eigen'
    elif runtype =='A':
        end = 'amp'
    deepest = int(abs(min(BtyLine)))+1
    file = f'{savepath}/{envfile}_time{texttime}_{end}.env'
    f = open(file, 'w')
    f.write(f"' {titleenv} ' \n")
    f.write(f'{freq} \n')
    f.write(f'{Nmedia} \n')
    f.write(f"'{topopt}' \n")
    f.write(f'50 0.00 {deepest} \n')
    f.write(f'    0.000000  1533  /\n')
    f.write(f'    {deepest}  1533  /\n')
    f.write(f"'{option2}'  0.0 \n")
    f.write(f' {deepest} {cpbottom} {csbottom} {den_bottom} {alpha_bottom} / ! lower halfspace \n')
    f.write(f'{NSD}			! NSD \n')
    f.write(f'{SD}  ! SD(1:NSD)  ... \n')
    f.write(f'{NRD} /		! NRD \n')
    f.write(f'{RD}   /		! RD(1:NRD)  ... \n')
    f.write(f'{NRR} /		! NRR \n')
    f.write(f'{RR}   /		! RR(1)  ... \n')
    f.write(f"'{runtype}' \n")
    f.write(f'{Nbeams} \n')
    f.write(f'{-alpha} {alpha} / \n')
    f.write(f'{step} {zbox} {rbox} \n')
    f.close()



In [75]:
for itime in range(1,3): #range(len(GPStime)):
    texttime = str(itime).zfill(3)
    savepath = f'time_{texttime}'
    if not os.path.isdir(savepath):
        os.makedirs(savepath)
    Ship_lat = 25+GPSlat['Ship'].values[itime]/60
    Ship_lon = 121+GPSlon['Ship'].values[itime]/60
    Buoy_lat = 25+GPSlat['Buoy'].values[itime]/60
    Buoy_lon = 121+GPSlon['Buoy'].values[itime]/60
    ## find the bty depth of the ship
    minlon = min(WD_new_lat_lon['lon'], key=lambda x:abs(x-Ship_lon))
    minlon_index = WD_new_lat_lon['lon'][WD_new_lat_lon['lon']==minlon].index[0]
    minlat = min(WD_new_lat_lon['lat'], key=lambda x:abs(x-Ship_lat))
    minlat_index = WD_new_lat_lon['lat'][WD_new_lat_lon['lat']==minlat].index[0]
    Shipbty = WD_bty[minlon_index][minlat_index]
    print(f'Ship depth: {Shipbty}m')
    ## find the depth of the Buoy
    minlon = min(WD_new_lat_lon['lon'], key=lambda x:abs(x-Buoy_lon))
    minlon_index = WD_new_lat_lon['lon'][WD_new_lat_lon['lon']==minlon].index[0]
    minlat = min(WD_new_lat_lon['lat'], key=lambda x:abs(x-Buoy_lat))
    minlat_index = WD_new_lat_lon['lat'][WD_new_lat_lon['lat']==minlat].index[0]
    Buoybty = WD_bty[minlon_index][minlat_index]
    print(f'Buoy depth: {Buoybty}m')

    BtyLine = [] ; SourceDist=[]
    for i,lon in enumerate(WD_new_lat_lon['lon']):
        for j,lat in enumerate(WD_new_lat_lon['lat']):
            if Buoy_lon<=lon<=Ship_lon and Buoy_lat<=lat<=Ship_lat:
            #    print(lon, lat, bty)
                bty = WD_bty[i][j]
                BtyLine.append(bty)
                dist,az,baz = gps2dist_azimuth(Ship_lat,Ship_lon,lat,lon)
                SourceDist.append(round(dist/1000,5))

    Dist_array = np.array(SourceDist)
    Bty_array = np.array(BtyLine)
    inds = Dist_array.argsort()
    sorted_Bty = Bty_array[inds]
    sorted_Dis = sorted(Dist_array)

    print(f'Max distance: {max(sorted_Dis)}km')

    if len(sorted_Dis)<20:
        qqq = 1 
    elif 20<=len(sorted_Dis)<100:
        qqq=4
    elif 100<=len(sorted_Dis)<200:
        qqq=10
    elif 200<len(sorted_Dis):
        qqq=20
    
    lentobty = int(len(sorted_Dis)/qqq)+2
    titleenv = f'ChaoJing {envfile} {texttime}'
    NSD = 1 ##number of sourece depth
    SD = 10 ##source depth
    NRD = 1 ##number of receiver depth
    RD = 3 ##receiver depth
    NRR  = 1 ## the number of receiver ranges
    RR = max(sorted_Dis) ##receiver range
    rbox = max(sorted_Dis)+0.5

    createBTY(lentobty, Shipbty, sorted_Dis,sorted_Bty,qqq,'E',rbox, savepath, envfile, texttime)
    createBTY(lentobty, Shipbty, sorted_Dis,sorted_Bty,qqq,'A',rbox, savepath, envfile, texttime)


    createENV(savepath, envfile,texttime, titleenv,freq,Nmedia, topopt, BtyLine,option2,cpbottom, csbottom, den_bottom,alpha_bottom, \
        NSD,SD,NRD,RD,NRR,RR,'E',Nbeams,alpha,step,zbox,rbox)
    createENV(savepath, envfile,texttime, titleenv,freq,Nmedia, topopt, BtyLine,option2,cpbottom, csbottom, den_bottom,alpha_bottom, \
        NSD,SD,NRD,RD,NRR,RR,'A',Nbeams,alpha,step,zbox,rbox)

Ship depth: -33.694988m
Buoy depth: -26.463713m
Max distance: 0.36797km
Ship depth: -33.694988m
Buoy depth: -26.463713m
Max distance: 0.3664km


In [74]:
min(BtyLine)

-32.599055999999997